In [1]:
#Código del TFM de Víctor Gabaldón Moreno (vicgabal@ucm.es) sobre Análisis de la contaminación lumínica en Madrid usando imagenes desde el espacio
#Este código sirve para cruzar catálogos con las detecciones realizadas. Requiere tener los archivos de las detecciones ya hechos.

In [ ]:
import pandas as pd
from astropy.io import ascii
import photutils
import astropy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.figure as f
import astropy.io.fits as fits
from math import pi
import pandas
import scipy
from scipy import stats
from scipy.optimize import curve_fit
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
import lmfit as lm
from photutils import detect_threshold
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
from photutils import detect_sources
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import os,sys,glob
import matplotlib as mpl
from matplotlib.pyplot import figure, show, rc, grid, savefig
from matplotlib.ticker import MultipleLocator
import numpy as np
from pylab import *
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from astropy.stats import sigma_clipped_stats
from astropy.stats import SigmaClip
from photutils import Background2D, MedianBackground
from astropy.stats import sigma_clipped_stats
from photutils import datasets
from photutils import DAOStarFinder
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture
from astropy.wcs import WCS

dpi=250
plt.rcParams['figure.dpi']= dpi

#%matplotlib qt

In [ ]:
#IDENTIFICACIÓN PARA TODAS LAS LISTAS DE FUENTES CON TODA LA BASE DE DATOS.
#Añade a cada elemento una casilla con la identificación en base a un criterio.

directory_load_Bases_de_datos='.\Bases_de_datos\\'
#nota: la base de datos mide la longitud positivamente hacia el ESTE
#nota 2: igual en en caso de las fuentes detectadas

directory_load_fits='.\Ficheros_fits_test\\'
contador_ext=0
files= [j for j in os.listdir(directory_load_fits)]

for index, file in enumerate(files):
 

    try:
        
        # tabla = pd.read_csv('.\Fuentes_puntuales\\Corr_iss062e102209B4F.cal2N.Ao2_rect.fits.csv',sep=';')
        tabla = pd.read_csv('.\Fuentes_puntuales\\'+str(file)+'.csv',sep=';') #file es el archivo del canal. Hay 4 por ahora ERROR
        
        #calculo de la r para el fits correspondiente a la imagen
        wcosa = WCS(directory_load_fits+file)
        lon0, lat0 = wcosa.all_pix2world(0, 0, 0)
        lon0=lon0-360
        lat0=lat0+0
        lon1, lat1 = wcosa.all_pix2world(0, 1, 0)
        lon1=lon1-360
        lat1=lat1+0
        restalon=lon0-lon1
        restalat=lat0-lat1
        r=4*np.sqrt(restalon**2+restalat**2) #r es 4 veces 1 pixel en coordenadas de lon
        print('r= '+str(r))

        
        w=np.arange(0,len(tabla['lon']),1)
        tabla['lugar']=''
    
        contador_ext=contador_ext+1
        print('\r', str(contador_ext)+' trabajando', sep='', end='', flush=True)

#########################################################################################################################################

        contador_in=0
        files_bases_datos= [i for i in os.listdir(directory_load_Bases_de_datos)]
        for index, file_bases_datos in enumerate(files_bases_datos):
            tabla_base_datos=pd.read_table('.\Bases_de_datos\\'+str(file_bases_datos), sep=";", names=["lon_lugar", "lat_lugar", "lugar"])

            #Calculo de la condición para que una fuente sea un elemento: las coordenadas del elemento tienen que estar dentro de un radio r<4 pixeles entorno a la fuente para la identificación
            #bug fixed

            #aplicación del criterio para reconocer N tipos en una imagen (la de banda azul)

            v=np.arange(0,len(tabla_base_datos['lon_lugar']),1) #array n=0,1,2,3,4,5...

            #m= elemento en la tabla de fuentes detectadas.
            #n= elemento en la tabla de puntos de interés
            #tabla.loc[fila,columna]

            for m in w:
            
                for n in v:
                    lugar_lon=tabla_base_datos.loc[n,'lon_lugar']
                    lugar_lat=tabla_base_datos.loc[n,'lat_lugar'] #coordenada de latitud del lugar n
                    lugar_lon=float(lugar_lon.replace(',','.')) #conversión del string (en la tabla viene como string ejemplo: 1,178 y lo convertimos a 1.178) a float
                    lugar_lat=float(lugar_lat.replace(',','.')) #conversión igual
    
                    r_lugar=np.sqrt((lugar_lon-tabla.loc[m,'lon'])**2+(lugar_lat-tabla.loc[m,'lat'])**2) #raiz[(longitud_lugar-longitud_puntos)^2+(latitud_lugar-latitud_puntos)^2]
    
                    if r_lugar<r:
                
                        tabla.loc[m,'lugar']=tabla.loc[m,'lugar']+','+tabla_base_datos.loc[n,'lugar'] #AÑADIR LA COLUMNA lugar a la tabla
    #                 print('ENCONTRADO ELEMENTO CON r_lugar= '+str(r_lugar))
    #                 print(tabla_base_datos.loc[n,'lugar'])
            
            tabla.to_csv('.\Identificacion\\'+str(file)+'.csv',sep=";",index=False)

            contador_in=contador_in+1
            completado_in=round(((contador_in/(50))*100), 2)
            print('\r', str(completado_in)+'% Completado de este canal', sep='', end='', flush=True)      
            
            
    except FileNotFoundError:
        pass

In [ ]:
#IDENTIFICACIÓN PARA LA REFERENCIA VERDE CON TODA LA BASE DE DATOS. Corr_iss062e102209G2F.cal2N.Ao2_rect.fits
#Programa auxiliar que sólo realiza identificaciones para la imagen de referencia en el canal verde anterior.

directory_load_Bases_de_datos='.\Bases_de_datos\\'
#nota: la base de datos mide la longitud positivamente hacia el ESTE
#nota 2: igual en en caso de las fuentes detectadas

directory_load_fits='.\Ficheros_fits_test\\'
contador_ext=0

file='Corr_iss062e102209G2F.cal2N.Ao2_rect.fits'

try:
        
    tabla = pd.read_csv('.\Fuentes_puntuales_referencia_verde\\referenciaCorr_iss062e102209G2F.cal2N.Ao2_rect.fits.csv',sep=';')
#               
    #calculo de la r para el fits correspondiente a la imagen
    wcosa = WCS(directory_load_fits+file)
    lon0, lat0 = wcosa.all_pix2world(0, 0, 0)
    lon0=lon0-360
    lat0=lat0+0
    lon1, lat1 = wcosa.all_pix2world(0, 1, 0)
    lon1=lon1-360
    lat1=lat1+0
    restalon=lon0-lon1
    restalat=lat0-lat1
    r=4*np.sqrt(restalon**2+restalat**2) #r es 4 veces 1 pixel en coordenadas de lon
    print('r= '+str(r))

        
    w=np.arange(0,len(tabla['lon']),1)
    tabla['lugar']=''
    
    contador_ext=contador_ext+1

#########################################################################################################################################

    contador_in=0
    files_bases_datos= [i for i in os.listdir(directory_load_Bases_de_datos)]
    for index, file_bases_datos in enumerate(files_bases_datos):
        tabla_base_datos=pd.read_table('.\Bases_de_datos\\'+str(file_bases_datos), sep=";", names=["lon_lugar", "lat_lugar", "lugar"])

        #Calculo de la condición para que una fuente sea un elemento: las coordenadas del elemento tienen que estar dentro de un radio r<4 pixeles entorno a la fuente para la identificación
        #bug fixed

        #aplicación del criterio para reconocer N tipos en una imagen (la de banda azul)

        v=np.arange(0,len(tabla_base_datos['lon_lugar']),1) #array n=0,1,2,3,4,5...

        #m= elemento en la tabla de fuentes detectadas.
        #n= elemento en la tabla de puntos de interés
        #tabla.loc[fila,columna]

        for m in w:
            
            for n in v:
                lugar_lon=tabla_base_datos.loc[n,'lon_lugar']
                lugar_lat=tabla_base_datos.loc[n,'lat_lugar'] #coordenada de latitud del lugar n
                lugar_lon=float(lugar_lon.replace(',','.')) #conversión del string (en la tabla viene como string ejemplo: 1,178 y lo convertimos a 1.178) a float
                lugar_lat=float(lugar_lat.replace(',','.')) #conversión igual
    
                r_lugar=np.sqrt((lugar_lon-tabla.loc[m,'lon'])**2+(lugar_lat-tabla.loc[m,'lat'])**2) #raiz[(longitud_lugar-longitud_puntos)^2+(latitud_lugar-latitud_puntos)^2]
    
                if r_lugar<r:
                
                    tabla.loc[m,'lugar']=tabla.loc[m,'lugar']+','+tabla_base_datos.loc[n,'lugar'] #AÑADIR LA COLUMNA lugar a la tabla
    #                print('ENCONTRADO ELEMENTO CON r_lugar= '+str(r_lugar))
    #                print(tabla_base_datos.loc[n,'lugar'])
            
        tabla.to_csv('.\Identificacion\\'+str(file)+'.csv',sep=";",index=False)

        contador_in=contador_in+1
        completado_in=round(((contador_in/(50))*100), 2)
        print('\r', str(completado_in)+'% Completado de este canal', sep='', end='', flush=True)      
            
            
except FileNotFoundError:
    pass

In [ ]:
#IDENTIFICACIÓN PARA LA REFERENCIA VERDE CON LA BASE DE DATOS EN CRISIS. Corr_iss062e102209G2F.cal2N.Ao2_rect.fits
#Herramienta que sólo realiza identificaciones para las categorías restringidas en caso de cierre total de Madrid.

directory_load_Bases_de_datos='.\Bases_de_datos_crisis\\'
#nota: la base de datos mide la longitud positivamente hacia el ESTE
#nota 2: igual en en caso de las fuentes detectadas

directory_load_fits='.\Ficheros_fits_test\\'
contador_ext=0

file='Corr_iss062e102209G2F.cal2N.Ao2_rect.fits'

try:
        
    tabla = pd.read_csv('.\Fuentes_puntuales_referencia_verde\\referenciaCorr_iss062e102209G2F.cal2N.Ao2_rect.fits.csv',sep=';')
#               
    #calculo de la r para el fits correspondiente a la imagen
    wcosa = WCS(directory_load_fits+file)
    lon0, lat0 = wcosa.all_pix2world(0, 0, 0)
    lon0=lon0-360
    lat0=lat0+0
    lon1, lat1 = wcosa.all_pix2world(0, 1, 0)
    lon1=lon1-360
    lat1=lat1+0
    restalon=lon0-lon1
    restalat=lat0-lat1
    r=4*np.sqrt(restalon**2+restalat**2) #r es 4 veces 1 pixel en coordenadas de lon
    print('r= '+str(r))

        
    w=np.arange(0,len(tabla['lon']),1)
    tabla['lugar']=''
    
    contador_ext=contador_ext+1

#########################################################################################################################################

    contador_in=0
    files_bases_datos= [i for i in os.listdir(directory_load_Bases_de_datos)]
    for index, file_bases_datos in enumerate(files_bases_datos):
        tabla_base_datos=pd.read_table('.\Bases_de_datos_crisis\\'+str(file_bases_datos), sep=";", names=["lon_lugar", "lat_lugar", "lugar"])

        #Calculo de la condición para que una fuente sea un elemento: las coordenadas del elemento tienen que estar dentro de un radio r<4 pixeles entorno a la fuente para la identificación
        #bug fixed

        #aplicación del criterio para reconocer N tipos en una imagen (la de banda azul)

        v=np.arange(0,len(tabla_base_datos['lon_lugar']),1) #array n=0,1,2,3,4,5...

        #m= elemento en la tabla de fuentes detectadas.
        #n= elemento en la tabla de puntos de interés
        #tabla.loc[fila,columna]

        for m in w:
            
            for n in v:
                lugar_lon=tabla_base_datos.loc[n,'lon_lugar']
                lugar_lat=tabla_base_datos.loc[n,'lat_lugar'] #coordenada de latitud del lugar n
                lugar_lon=float(lugar_lon.replace(',','.')) #conversión del string (en la tabla viene como string ejemplo: 1,178 y lo convertimos a 1.178) a float
                lugar_lat=float(lugar_lat.replace(',','.')) #conversión igual
    
                r_lugar=np.sqrt((lugar_lon-tabla.loc[m,'lon'])**2+(lugar_lat-tabla.loc[m,'lat'])**2) #raiz[(longitud_lugar-longitud_puntos)^2+(latitud_lugar-latitud_puntos)^2]
    
                if r_lugar<r:
                
                    tabla.loc[m,'lugar']=tabla.loc[m,'lugar']+','+tabla_base_datos.loc[n,'lugar'] #AÑADIR LA COLUMNA lugar a la tabla
    #                print('ENCONTRADO ELEMENTO CON r_lugar= '+str(r_lugar))
    #                print(tabla_base_datos.loc[n,'lugar'])
            
        tabla.to_csv('.\Identificacion_crisis\\'+str(file)+'.csv',sep=";",index=False)

        contador_in=contador_in+1
        completado_in=round(((contador_in/(7))*100), 2)
        print('\r', str(completado_in)+'% Completado de este canal', sep='', end='', flush=True)      
            
            
except FileNotFoundError:
    pass